In [4]:
import tkinter as tk
import datetime
from dateutil import parser
from pymongo import MongoClient
from passlib.context import CryptContext
from pprint import pprint

In [5]:
client = MongoClient(port=27017)
db = client.TransportDept
pwd_context = CryptContext(schemes=["sha512_crypt"], 
                           default="sha512_crypt", 
                           sha512_crypt__default_rounds=45000)

In [18]:
#create a new booking
def create_new_booking(date,no_people,eid,hiretime,destination,distance,waitingtime,reasonhire,cartype):
    if cartype>db.regemployee.find_one({'eid':eid})['Car Type']:
        return "You dont have authorization to book this car type"
    if hiretime<=0:
        return "Hire Time cannot be 0 or less than 0"
    if date<=datetime.datetime.now():
        return "Incorrect Date Time entered"
    car_av = db.cars.find({'Car Type':cartype})
    car_for_booking=0
    for c in car_av:
        if c['No Of Seats']<no_people:
            continue
        flag = True
        for i in range(hiretime):#0 index cause time is one hour is counted as say 11-12 entry has 11
            if (date + datetime.timedelta(hours=date.hour+i) in c['Calendar']):
                flag=False
                break
        if flag==True:
            car_for_booking=c
            break
    
    if car_for_booking==0:
        return "Car not available"
    
    driver_available=0
    for d in db.driver.find():
        flag=True
        for i in range(hiretime):#0 index cause time is one hour is counted as say 11-12 entry has 11
            if (date + datetime.timedelta(hours=date.hour+i) in d['Calendar']):
                flag=False
                break
        if(flag==True):
            driver_available=d
            break
    
    if driver_available==0:
        return "No drivers are available for this time"
    
    #update cars,driver and carhiredb
    for i in range(hiretime):#0 index cause time is one hour is counted as say 11-12 entry has 11
        db.driver.update_one({'eid': driver_available['eid']}, {'$push': {'Calendar':date + datetime.timedelta(hours=i)}})
        db.cars.update_one({'Reg No': car_for_booking['Reg No']}, {'$push': {'Calendar':date + datetime.timedelta(hours=i)}})
        
    car_hire = {'Car Reg No':car_for_booking['Reg No'],
          'Date Time':date,
          'Driver Eid':driver_available['eid'],
          'Employee Eid':eid,
          'Reason of Hire':reasonhire,
          'Fuel Consumed':"",
          'Expenditure':"",
          'Distance':distance,
          'Destination':destination,
          'No of People':no_people,
          'Hire Time':hiretime,
          'Waiting Time':0,
          'Approved':1}
    
    db.carhire.insert_one(car_hire)
    return "Trip Created"
            
            
    
    
        

In [19]:
eid=0

class SampleApp(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self._frame = None
        self.switch_frame(HomePage)

    def switch_frame(self, frame_class):
        """Destroys current frame and replaces it with a new one."""
        new_frame = frame_class(self)
        if self._frame is not None:
            self._frame.destroy()
        self._frame = new_frame
        self._frame.pack()


class HomePage(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)

        start_label = tk.Label(self, text="Transport Department\n")
        signin_button = tk.Button(self, text="Sign In",
                                  command=lambda: master.switch_frame(SignIn))
        start_label.pack(side="top", fill="x", pady=10)
        signin_button.pack()


class SignIn(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)

        start_label = tk.Label(self, text="Enter the following credentials")
        
        eid_label = tk.Label(self, text="Employee Id")
        self.eid_entry = tk.Entry(self)
        password_label = tk.Label(self, text="Password")
        self.password_entry = tk.Entry(self,show="*")
        start_button = tk.Button(self, text="Login",
                                 command=lambda: self.callback(master))
        start_label.pack(side="top", fill="x", pady=10)
        eid_label.pack()
        self.eid_entry.pack()
        password_label.pack()
        self.password_entry.pack()
        start_button.pack()
    def callback(self,master):
        global eid
        eid=self.eid_entry.get()
        pwd=self.password_entry.get()
        master.switch_frame(UserMenu)
class UserMenu(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)

        start_label = tk.Label(self, text="Welcome\n")
        Book_button = tk.Button(self, text="Book a car",
                                  command=lambda: master.switch_frame(Booking))
        VC_button = tk.Button(self, text="View/Cancel",
                                  command=lambda: master.switch_frame(VCPage))
        Logout_button = tk.Button(self, text="Logout",
                                  command=lambda: master.switch_frame(HomePage))
        alert_label = tk.Label(self, text="Alerts\n")
        start_label.pack(side="top", fill="x", pady=10)
        Book_button.pack()
        VC_button.pack()
        Logout_button.pack()
        alert_label.pack()


class Booking(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)

        start_label = tk.Label(self, text="Enter the following -")
        
        date_label = tk.Label(self, text="Date(DD-MM-YYY)")
        self.date_entry = tk.Entry(self)
        time_label = tk.Label(self, text="Time(in hours from 0-23)")
        self.time_entry = tk.Entry(self)
        noofpeople_label = tk.Label(self, text="No. of people")
        self.noofpeople_entry = tk.Entry(self)
        hiretime_label = tk.Label(self, text="Hire Time")
        self.hiretime_entry = tk.Entry(self)
        destination_label = tk.Label(self, text="Destination")
        self.destination_entry = tk.Entry(self)
        distance_label = tk.Label(self, text="Distance")
        self.distance_entry = tk.Entry(self)
        waitingtime_label = tk.Label(self, text="Waiting time")
        self.waitingtime_entry = tk.Entry(self)
        reason_label = tk.Label(self, text="Reason for hire")
        self.reason_entry = tk.Entry(self)
        ctype_label = tk.Label(self, text="Car type")
        self.ctype_entry = tk.Entry(self)
        
        start_button = tk.Button(self, text="Book",
                                 command=lambda: self.callback(master))
        start_label.pack(side="top", fill="x", pady=10)
        
        date_label.pack()
        self.date_entry.pack()
        time_label.pack()
        self.time_entry.pack()
        noofpeople_label.pack()
        self.noofpeople_entry.pack()
        hiretime_label.pack()
        self.hiretime_entry.pack()
        destination_label.pack()
        self.destination_entry.pack()
        distance_label.pack()
        self.distance_entry.pack()
        waitingtime_label.pack()
        self.waitingtime_entry.pack()
        reason_label.pack()
        self.reason_entry.pack()
        ctype_label.pack()
        self.ctype_entry.pack()
        start_button.pack()
        
    def callback(self,master):
        date=self.date_entry.get()
        #print(eid)
        
        
        time=self.time_entry.get()
        dt=parser.parse(date,dayfirst=True) + datetime.timedelta(hours=int(time))
        print(dt)
        
        noofpeople=self.noofpeople_entry.get()
        
        hiretime=self.hiretime_entry.get()
        
        destination=self.destination_entry.get()
        
        distance=self.distance_entry.get()
        
        waitingtime=self.waitingtime_entry.get()
        
        reason=self.reason_entry.get()
        
        ctype=self.ctype_entry.get()
        msg=create_new_booking(dt,int(noofpeople),int(eid),int(hiretime),destination,int(distance),int(waitingtime),reason,int(ctype))
        print(msg)
        master.switch_frame(UserMenu)
class VCPage(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)
        
        
        start_label = tk.Label(self, text="View/Cancel")
        
        cancel_button = tk.Button(self, text="Cancel",command=lambda: self.cancel(master))
        back_button = tk.Button(self, text="Back",command=lambda: master.switch_frame(UserMenu))
        self.booking1_checkbutton =tk.Checkbutton(self,text="Booking 1")
        self.booking2_checkbutton =tk.Checkbutton(self,text="Booking 2")
        
        start_label.pack(side="top", fill="x", pady=10)
        back_button.pack(side="bottom", fill="x", pady=10)
        cancel_button.pack(side="bottom", fill="x", pady=10)
        #if(var.get()==0):
        self.booking1_checkbutton.pack()
        self.booking2_checkbutton.pack()
        
    def cancel(self,master):
        #print(var.get())
        master.switch_frame(VCPage)


if __name__ == "__main__":
    app = SampleApp()
    app.mainloop()

2018-05-15 12:00:00
Trip Created
